# Random Forest from scratch

In [6]:
import numpy as np
from collections import Counter
# we need nbimporter to import functions or classes from another jupyter notebook
# for myself, it's installed in base environment!
import nbimporter
import DecisionTree_from_scratch
from DecisionTree_from_scratch import DecisionTree

In [13]:
def bootstrap_sample(X, y):
    n_samples = X.shape[0]
    idxs = np.random.choice(n_samples, n_samples, replace=True)
    return X[idxs], y[idxs]

def most_common_label(y):
    counter = Counter(y)
    # it returns a list of key-values, we pick the first pair, and choose the key! thus [0][0]
    most_common = counter.most_common(1)[0][0]
    return most_common

class RandomForest:
    def __init__(self, n_trees=10, min_samples_split=2, max_depth=100, n_feats=None):
        self.n_trees = n_trees
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.trees = []
    
    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTree(min_samples_split=self.min_samples_split, max_depth=self.max_depth, n_feats=self.n_feats)
            X_samp, y_samp = bootstrap_sample(X, y)
            tree.fit(X_samp, y_samp)
            self.trees.append(tree)
    
    def predict(self, X):
        preds = np.array([tree.predict(X) for tree in self.trees])
        # original order: each row represents predictions for all samples X using a single tree
        # swap axes
        # desired order: each row represents predictions for a single sample x using all trees
        preds = preds.swapaxes(0, 1)
        y_pred = [most_common_label(pred) for pred in preds]
        return y_pred
            

In [14]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

dataset = datasets.load_breast_cancer()
X, y = dataset.data, dataset.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

clf = RandomForest(n_trees=2, max_depth=5)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
acc = accuracy(y_pred, y_test)

print('Accuracy: ', acc)

Accuracy:  0.9122807017543859
